In [22]:
import os
import pandas as pd
from pathlib import Path
import json
import cv2
dataset_root = 'action_dataset'
sample_rate = 20

# Create annotation file for swin transformer

In [12]:
df = pd.read_csv(Path(dataset_root, 'dance-train.csv'))
classes = df['label'].unique()
class_mapper = {class_name: i for i, class_name in enumerate(classes)}
with open(Path(dataset_root, 'class_mapper.json'), 'w') as f:
    json.dump(class_mapper, f)

In [15]:
df = pd.read_csv(Path(dataset_root, 'dance-train.csv'))
videos = os.listdir(Path(dataset_root, 'videos'))
with open(Path(dataset_root, 'train_annotation.txt'), 'w') as f:
    for _, row in df.iterrows():
        if row["youtube_id"] + ".mp4" not in videos:
            continue
        annotation = f'{row["youtube_id"] + ".mp4"} {class_mapper[row["label"]]}\n'
        f.write(annotation)

In [16]:
df = pd.read_csv(Path(dataset_root, 'dance-validate.csv'))
with open(Path(dataset_root, 'validate_annotation.txt'), 'w') as f:
    for _, row in df.iterrows():
        if row["youtube_id"] + ".mp4" not in videos:
            continue
        annotation = f'{row["youtube_id"] + ".mp4"} {class_mapper[row["label"]]}\n'
        f.write(annotation)

# Create frames from train videos for 2d net

In [18]:
df = pd.read_csv(Path(dataset_root, 'dance-train.csv'))

In [ ]:
for _, row in df.iterrows():
    if row["youtube_id"] + ".mp4" not in videos:
        continue
    cap = cv2.VideoCapture(str(Path(dataset_root, 'videos', row["youtube_id"] + ".mp4")))
    success, img = cap.read()
    fno = 1
    while success:
        label = row["label"]
        imgs_dir = Path(dataset_root, 'imgs', label)
        os.makedirs(imgs_dir, exist_ok=True)
        if fno % sample_rate == 0:
            cv2.imwrite(str(imgs_dir / f'{row["youtube_id"]}_{fno}.png'), img)
        
        # read next frame
        success, img = cap.read()
        fno += 1
    print(row["youtube_id"])

# Pritnt 2d net metrics

In [44]:
import json
from sklearn.metrics import top_k_accuracy_score, classification_report

with open("test_results.json", "r") as f:
    gt, preds = json.load(f)

In [55]:
print(classification_report(gt, preds))

              precision    recall  f1-score   support

           0       0.67      0.65      0.66        46
           1       0.27      0.30      0.28        44
           2       0.20      0.18      0.19        45
           3       0.26      0.27      0.26        45
           4       0.24      0.23      0.24        43
           5       0.33      0.37      0.35        43
           6       0.38      0.37      0.38        43
           7       0.33      0.33      0.33        45
           8       0.37      0.37      0.37        43
           9       0.42      0.41      0.42        46
          10       0.32      0.33      0.32        46
          11       0.35      0.38      0.37        45
          12       0.32      0.28      0.30        46
          13       0.28      0.27      0.27        45
          14       0.43      0.28      0.34        46
          15       0.00      0.00      0.00         0

    accuracy                           0.34       671
   macro avg       0.32   

/home/user/.local/lib/python3.8/site-packages/scikit_learn-1.2.0-py3.8-linux-x86_64.egg/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/scikit_learn-1.2.0-py3.8-linux-x86_64.egg/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/user/.local/lib/python3.8/site-packages/scikit_learn-1.2.0-py3.8-linux-x86_64.egg/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.